## Model 1 - Location Prediction

In [2]:
import pandas as pd
import json
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn import linear_model as lm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import math

/home/ec2-user/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/ec2-user/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
#Ignore warnings 
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

## I. Data (wo Finer segregation in Asian Population)


In [4]:
raw_data = pd.read_csv('../../../data/phoenix_business_ws_rw_ffall_merged2.csv', skipinitialspace=True)

raw_data.head(1)

,zipcode,business_id,Mexican,American (Traditional),Pizza,American (New),Burgers,Italian,Chinese,Salad,...,median_income,median_age,under_18,above_18,walkscore,ffall,stars,review_count,ffall_category,CuisineCombined
0,85007,_c3ixq9jYKxhLUB0czi0ug,0,0,0,0,1,0,0,0,...,27557,35.0,3652,8906,60.0,987,4.0,277,4,14


All the available attibutes in the given dataset. 

### 2. Model Building

#### 2.1. Data preparation

In [136]:
dframe = raw_data[(raw_data['ffall']<=888) & (raw_data['ffall']>=3)]
dframe['totalStars'] = dframe['review_count'] * dframe['stars']
dframe['adjwhp'] = dframe['white_pop'] * dframe['stars']
dframe['adjpafp'] = dframe['afam_pop'] * dframe['stars']
dframe['adjindp'] = dframe['amindian_pop'] * dframe['stars']
dframe['adjasp'] = dframe['asian_pop'] * dframe['stars']
dframe['adjhwp'] = dframe['hawaiian_pop'] * dframe['stars']
dframe['adjorp'] = dframe['other_race'] * dframe['stars']

train, test = train_test_split(dframe, test_size=0.05)
dframe.head(1)

/home/ec2-user/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ec2-user/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ec2-user/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,zipcode,business_id,Mexican,American (Traditional),Pizza,American (New),Burgers,Italian,Chinese,Salad,...,review_count,ffall_category,CuisineCombined,totalStars,adjwhp,adjpafp,adjindp,adjasp,adjhwp,adjorp
1,85033,3ByGQOVgds2YEu6kzl-XEQ,0,1,0,0,0,0,0,0,...,3,0,11,7.5,72047.5,6790.0,2300.0,1110.0,167.5,45272.5


In [137]:
zip_means_df = dframe.groupby(['zipcode']).agg([np.mean])
zip_avg_ffall_revC_list = []
rowitem = []

for ind, row in zip_means_df.iterrows():
    zip_avg_ffall_revC_list.append([ind, row.iloc[65], row.iloc[63], row[66]])
    
zip_avg_ffall_revC_df = pd.DataFrame(zip_avg_ffall_revC_list, columns=['zipcode','avgrc','avgffall', 'avgffc'])
zip_avg_ffall_revC_df.drop_duplicates(inplace = True)

In [38]:
X_train = train.drop(columns=['zipcode','business_id','ffall_category','CuisineCombined','total_pop','male','female','under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 'totalStars', 'zipcode.1', 'median_age', 'walkscore'])
y_train = train['ffall']

X_train.head()
y_train.head()

X_test = test.drop(columns=['zipcode','business_id','ffall_category','CuisineCombined','total_pop','male','female','under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 'totalStars', 'zipcode.1','median_age', 'walkscore'])
y_test = test['ffall']

In [32]:
X_train.columns

Index(['Mexican', 'American (Traditional)', 'Pizza', 'American (New)',
       'Burgers', 'Italian', 'Chinese', 'Salad', 'Sports Bars', 'Seafood',
       'Japanese', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Asian Fusion',
       'Steakhouses', 'Greek', 'Tex-Mex', 'Thai', 'Vietnamese', 'Indian',
       'Middle Eastern', 'Southern', 'Latin American', 'Hawaiian', 'Korean',
       'French', 'Caribbean', 'Pakistani', 'Ramen', 'New Mexican Cuisine',
       'Modern European', 'Spanish', 'African', 'Cantonese', 'Persian/Iranian',
       'Filipino', 'Cuban', 'Mongolian', 'Lebanese', 'Polish', 'Taiwanese',
       'German', 'Turkish', 'Ethiopian', 'Brazilian', 'Afghan', 'white_pop',
       'afam_pop', 'amindian_pop', 'asian_pop', 'hawaiian_pop', 'other_race',
       'median_income'],
      dtype='object')

#### 2.2. Basic Model - Random Forrest Model

In [233]:
from sklearn.ensemble import RandomForestRegressor

In [272]:
X_train.columns

Index(['Mexican', 'American (Traditional)', 'Pizza', 'American (New)',
       'Burgers', 'Italian', 'Chinese', 'Salad', 'Sports Bars', 'Seafood',
       'Japanese', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Asian Fusion',
       'Steakhouses', 'Greek', 'Tex-Mex', 'Thai', 'Vietnamese', 'Indian',
       'Middle Eastern', 'Southern', 'Latin American', 'Hawaiian', 'Korean',
       'French', 'Caribbean', 'Pakistani', 'Ramen', 'New Mexican Cuisine',
       'Modern European', 'Spanish', 'African', 'Cantonese', 'Persian/Iranian',
       'Filipino', 'Cuban', 'Mongolian', 'Lebanese', 'Polish', 'Taiwanese',
       'German', 'Turkish', 'Ethiopian', 'Brazilian', 'Afghan', 'white_pop',
       'afam_pop', 'amindian_pop', 'asian_pop', 'hawaiian_pop', 'other_race',
       'median_income'],
      dtype='object')

In [273]:
rf = RandomForestRegressor(n_estimators = 10, random_state = 42, max_depth=8)

In [274]:
rf.fit(X_train, y_train);
predictions = rf.predict(X_train)
errors = abs(predictions - y_train)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 133.51 degrees.


In [275]:
#RSS calculation
y_mean = np.mean(y_train)

ssr = np.sum((predictions-y_train)**2)
sst = np.sum((y_train-y_mean)**2)

print(ssr)
print(sst)

r = 1- ssr/sst

print(r)

# Calculate mean absolute percentage error (MAPE)
#mape = 100 * (errors / y_test)
# Calculate and display accuracy
#accuracy = 100 - np.mean(mape)
#print('Accuracy:', round(accuracy, 2), '%.')

68315914.25448632
94938464.57887454
0.2804190108032588


In [276]:
rf.feature_importances_*100

array([ 1.88298119,  4.3132331 ,  1.56510945,  3.35378666,  2.9164051 ,
        1.11986514,  1.57203702,  6.30585182,  1.98351477,  1.46133206,
        0.20569031,  2.38717008,  0.74421925,  1.27540962,  1.98493778,
        4.44545877,  0.69845513,  0.7740141 ,  3.55782597,  1.84598829,
        0.12319883,  0.38164754,  0.24941129,  1.03783167,  0.33463572,
        0.46094546,  1.1749527 ,  0.74943558,  0.66755936,  0.        ,
        0.69356502,  0.16764701,  0.0306361 ,  0.        ,  0.03010604,
        0.19497642,  1.35857985,  0.        ,  0.56414667,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.02705986,  3.49224872,  8.63456553,  4.85948138,
        4.87341616,  5.58270734, 13.41592255,  6.50203762])

#### 2.3. Tune the model

In [109]:
params = [{'n_estimators': [10, 11, 12, 13, 14, ], 'max_depth': [5, 6, 7] }]
clf = GridSearchCV(RandomForestRegressor(), params, cv=10)
clf.fit(X_train, y_train)
print("Random forest classifier - Grid Search - Best params: ", clf.best_params_)
print("Random forest classifier - Grid Search - Best Score: ", clf.best_score_)

Random forest classifier - Grid Search - Best params:  {'max_depth': 5, 'n_estimators': 11}
Random forest classifier - Grid Search - Best Score:  0.05672110438667231


#### 2.4. Fit the tuned model

In [239]:
rf = RandomForestRegressor(n_estimators = 11, random_state = 42, max_depth=5)
rf.fit(X_train, y_train);
preds = rf.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 145.101112


#### 2.5. Basic Model - XGBoost

In [47]:
import xgboost as xgb

In [48]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [241]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 155.806330


In [280]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

params = [{'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [25, 30, 35], 'max_depth': [5, 10, 15], 
           'colsample_bytree': [0.3, 0.6, 0.9], 'objective': ['reg:linear'], 'alpha': [10]}]

clf = GridSearchCV(xgb.XGBRegressor(), params, cv=10)

clf.fit(X_train_sc, y_train)
print("Random forest classifier - Grid Search - Best params: ", clf.best_params_)
print("Random forest classifier - Grid Search - Best Score: ", clf.best_score_)

Random forest classifier - Grid Search - Best params:  {'alpha': 10, 'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 30, 'objective': 'reg:linear'}
Random forest classifier - Grid Search - Best Score:  0.0873763292603219


#### 2.6. Fit the tuned model

In [242]:
# The following set of parameters give the best performance
# {'alpha': 10, 'colsample_bytree': 0.3, 'learning_rate': 0.1, 
#  'max_depth': 5, 'n_estimators': 30, 'objective': 'reg:linear'}
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 35)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 146.013344


#### 2.8. Just population dataset

In [179]:
X_tr_pop = train[['white_pop', 'afam_pop', 'amindian_pop', 'asian_pop', 'hawaiian_pop', 'other_race', 
                  'median_income', 'walkscore']]
X_tr_cu = train[['Mexican', 'American (Traditional)', 'Pizza', 'American (New)',
       'Burgers', 'Italian', 'Chinese', 'Salad', 'Sports Bars', 'Seafood',
       'Japanese', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Asian Fusion',
       'Steakhouses', 'Greek', 'Tex-Mex', 'Thai', 'Vietnamese', 'Indian',
       'Middle Eastern', 'Southern', 'Latin American', 'Hawaiian', 'Korean',
       'French', 'Caribbean', 'Pakistani', 'Ramen', 'New Mexican Cuisine',
       'Modern European', 'Spanish', 'African', 'Cantonese', 'Persian/Iranian',
       'Filipino', 'Cuban', 'Mongolian', 'Lebanese', 'Polish', 'Taiwanese',
       'German', 'Turkish', 'Ethiopian', 'Brazilian', 'Afghan']]
y_tr = train['ffall']

X_te_pop = test[['white_pop', 'afam_pop', 'amindian_pop', 'asian_pop', 'hawaiian_pop', 'other_race', 
                  'median_income', 'walkscore']]
X_te_cu = test[['Mexican', 'American (Traditional)', 'Pizza', 'American (New)',
       'Burgers', 'Italian', 'Chinese', 'Salad', 'Sports Bars', 'Seafood',
       'Japanese', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Asian Fusion',
       'Steakhouses', 'Greek', 'Tex-Mex', 'Thai', 'Vietnamese', 'Indian',
       'Middle Eastern', 'Southern', 'Latin American', 'Hawaiian', 'Korean',
       'French', 'Caribbean', 'Pakistani', 'Ramen', 'New Mexican Cuisine',
       'Modern European', 'Spanish', 'African', 'Cantonese', 'Persian/Iranian',
       'Filipino', 'Cuban', 'Mongolian', 'Lebanese', 'Polish', 'Taiwanese',
       'German', 'Turkish', 'Ethiopian', 'Brazilian', 'Afghan']]
y_te = test['ffall']

In [180]:
X_train.columns

Index(['Mexican', 'American (Traditional)', 'Pizza', 'American (New)',
       'Burgers', 'Italian', 'Chinese', 'Salad', 'Sports Bars', 'Seafood',
       'Japanese', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Asian Fusion',
       'Steakhouses', 'Greek', 'Tex-Mex', 'Thai', 'Vietnamese', 'Indian',
       'Middle Eastern', 'Southern', 'Latin American', 'Hawaiian', 'Korean',
       'French', 'Caribbean', 'Pakistani', 'Ramen', 'New Mexican Cuisine',
       'Modern European', 'Spanish', 'African', 'Cantonese', 'Persian/Iranian',
       'Filipino', 'Cuban', 'Mongolian', 'Lebanese', 'Polish', 'Taiwanese',
       'German', 'Turkish', 'Ethiopian', 'Brazilian', 'Afghan', 'white_pop',
       'afam_pop', 'amindian_pop', 'asian_pop', 'hawaiian_pop', 'other_race',
       'median_income'],
      dtype='object')

#### 2.7 Just trying pure regression

In [43]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)

# add_constant has to be done by the user. Otherwise it will not add a intercept to the model
X2 = sm.add_constant(X_train_sc)
est = sm.OLS(np.log(y_train), X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  ffall   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     8.909
Date:                Mon, 19 Nov 2018   Prob (F-statistic):           3.70e-61
Time:                        05:24:43   Log-Likelihood:                -3981.0
No. Observations:                2318   AIC:                             8070.
Df Residuals:                    2264   BIC:                             8380.
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2792      0.028    151.064      0.0

In [49]:
model = lm.LinearRegression()
model.fit(X_train_sc, np.log(y_train))

X_test_sc = scaler.transform(X_test)

preds = model.predict(X_test_sc)

rmse = np.sqrt(mean_squared_error(np.log(y_test), preds))
print("RMSE: %f" % (rmse))

RMSE: 1.381602


In [50]:
# The following set of parameters give the best performance
# {'alpha': 10, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 
#  'max_depth': 5, 'n_estimators': 25, 'objective': 'reg:linear'}
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 30)

xg_reg.fit(X_train_sc, np.log(y_train))

preds = xg_reg.predict(X_test_sc)

rmse = np.sqrt(mean_squared_error(np.log(y_test), preds))
print("RMSE: %f" % (rmse))

RMSE: 1.414943


In [51]:
# add_constant has to be done by the user. Otherwise it will not add a intercept to the model
X2 = sm.add_constant(X_tr_pop)
est = sm.OLS(np.log(y_tr), X2)
est2 = est.fit()
print(est2.summary())

NameError: name 'X_tr_pop' is not defined

## II. Data (w Finer Segregation in Asian Population)

In [138]:
pop_data = pd.read_csv('../../../data/arizon.csv', skipinitialspace=True)

pop_data.head(1)

,Unnamed: 0,Unnamed: 0.1,median_income,housing_units,occupied_housing_units,total_pop,white_pop,afam_pop,amindian_pop,asian_pop,...,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000
0,0,0.0,46146.0,13246.0,11418.0,39507.0,20634.0,3754.0,731.0,971.0,...,441,186,78,76,52,28,19,2,0,0


In [139]:
selected_pop = pop_data[['zipcode', 'PCT0050002', 'PCT0050003', 'PCT0050004', 'PCT0050005', 'PCT0050006',
       'PCT0050007', 'PCT0050008', 'PCT0050009', 'PCT0050010', 'PCT0050011',
       'PCT0050012', 'PCT0050013', 'PCT0050014', 'PCT0050015', 'PCT0050016',
       'PCT0050017', 'PCT0050018', 'PCT0050019', 'PCT0050020', 'PCT0050021',
       'PCT0050022']]

In [140]:
joined_data = dframe.merge(selected_pop, left_on="zipcode", right_on="zipcode", how="inner", suffixes = ("_a","_b"))

In [141]:
final_data = joined_data.merge(zip_avg_ffall_revC_df, left_on="zipcode", right_on="zipcode", how="inner", suffixes = ("_a","_b"))
final_data['stars_avgffc'] = final_data['stars'] * final_data['avgffall']

In [142]:
final_data.columns

Index(['zipcode', 'business_id', 'Mexican', 'American (Traditional)', 'Pizza',
       'American (New)', 'Burgers', 'Italian', 'Chinese', 'Salad',
       ...
       'PCT0050017', 'PCT0050018', 'PCT0050019', 'PCT0050020', 'PCT0050021',
       'PCT0050022', 'avgrc', 'avgffall', 'avgffc', 'stars_avgffc'],
      dtype='object', length=102)

In [143]:
#dframe = final_data
train, test = train_test_split(final_data, test_size=0.05)

In [144]:
X_train = train.drop(columns=['zipcode','business_id', 'CuisineCombined','total_pop',
                              'male','female','under_18','above_18','occupied_housing_units', 'review_count', 
                              'stars', 'ffall', 'zipcode.1', 'median_age', 'zipcode.1', 
                              'asian_pop', 'avgrc', 'ffall_category', 'white_pop', 'afam_pop', 'amindian_pop', 
                              'hawaiian_pop', 'other_race'])
#dframe[['white_pop', 'afam_pop', 'amindian_pop', 'hawaiian_pop', 'other_race']]
y_train = train['ffall']

X_train.head()
y_train.head()

X_test = test.drop(columns=['zipcode','business_id','CuisineCombined','total_pop','male','female',
                            'under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 
                            'zipcode.1','median_age', 'zipcode.1', 'asian_pop', 'avgrc', 'ffall_category', 
                            'white_pop', 'afam_pop', 'amindian_pop', 'hawaiian_pop', 'other_race'])
y_test = test['ffall']

#### Fit a new XGBoost model

In [61]:
# The following set of parameters give the best performance
# {'alpha': 10, 'colsample_bytree': 0.3, 'learning_rate': 0.1, 
# 'max_depth': 5, 'n_estimators': 30, 'objective': 'reg:linear'}
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 30)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 1059.600022


In [63]:
params = [{'n_estimators': [15, 20, 50, 100, 200], 'max_depth': [5, 6, 10,15,20] }]
clf = GridSearchCV(RandomForestRegressor(), params, cv=10)
clf.fit(X_train, y_train)
print("Random forest classifier - Grid Search - Best params: ", clf.best_params_)
print("Random forest classifier - Grid Search - Best Score: ", clf.best_score_)

Random forest classifier - Grid Search - Best params:  {'max_depth': 5, 'n_estimators': 100}
Random forest classifier - Grid Search - Best Score:  0.07566492809529435


In [65]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth=5)

rf.fit(X_train, y_train)

preds = rf.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 1051.880935


## III. Linear Regression

In [192]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [193]:
# X is a numpy array with your features
# y is the label array
#enc = OneHotEncoder(sparse=False)
#X_tr_transform = enc.fit_transform(X_train)
#X_te_transform = enc.fit_transform(X_test)

# apply your linear regression as you want
#model = LinearRegression(fit_intercept = False)
#model.fit(X_tr_transform, y_train)

#preds = model.predict(X_te_transform)

#rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))

In [145]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

#Regression
lmod = lm.LinearRegression()
lmod.fit(X_train_sc, np.log(y_train))

preds = lmod.predict(X_test_sc)
rmse = np.sqrt(mean_squared_error(np.log(y_test), preds))
print("RMSE: %f" % (rmse))

RMSE: 0.997638


In [146]:
# add_constant has to be done by the user. Otherwise it will not add a intercept to the model
X2 = sm.add_constant(X_train_sc)
est = sm.OLS(np.log(y_train), X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  ffall   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     32.03
Date:                Mon, 19 Nov 2018   Prob (F-statistic):          2.69e-309
Time:                        07:07:28   Log-Likelihood:                -3304.7
No. Observations:                2318   AIC:                             6771.
Df Residuals:                    2237   BIC:                             7237.
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2914      0.021    201.607      0.0

In [ ]:
X_train_dummy = train.drop(columns=['zipcode','business_id', 'CuisineCombined','total_pop',
                              'male','female','under_18','above_18','occupied_housing_units', 'review_count', 
                              'stars', 'ffall', 'zipcode.1', 'median_age', 'zipcode.1', 
                              'asian_pop', 'avgrc', 'ffall_category'])
y_train = train['ffall']

X_train.head()
y_train.head()

X_test = test.drop(columns=['zipcode','business_id','CuisineCombined','total_pop','male','female',
                            'under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 
                            'zipcode.1','median_age', 'zipcode.1', 'asian_pop', 'avgrc', 'ffall_category'])
y_test = test['ffall']

In [151]:
y_test.describe()

count     142.000000
mean      424.739437
std       861.274535
min         1.000000
25%        18.000000
50%       100.500000
75%       411.750000
max      6007.000000
Name: ffall, dtype: float64

In [251]:
dframe2 = raw_data[raw_data['ffall']>600]
dframe2['totalStars'] = dframe2['review_count'] * dframe2['stars']
train2, test2 = train_test_split(dframe2, test_size=0.05)
dframe2['totalStars'].head()

X_train2 = train2.drop(columns=['zipcode','business_id','ffall_category','CuisineCombined','total_pop','male','female','under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 'totalStars', 'zipcode.1', 'median_age', 'walkscore'])
y_train2 = train2['ffall']

X_test2 = test2.drop(columns=['zipcode','business_id','ffall_category','CuisineCombined','total_pop','male','female','under_18','above_18','occupied_housing_units', 'review_count', 'stars', 'ffall', 'totalStars', 'zipcode.1','median_age', 'walkscore'])
y_test2 = test2['ffall']

scaler = StandardScaler()
scaler.fit(X_train2)
X_train_sc2 = scaler.transform(X_train2)


# add_constant has to be done by the user. Otherwise it will not add a intercept to the model
X3 = sm.add_constant(X_train_sc2)
est3 = sm.OLS(y_train2, X3)
est4 = est3.fit()
print(est4.summary())

                            OLS Regression Results                            
Dep. Variable:                  ffall   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     1.568
Date:                Mon, 19 Nov 2018   Prob (F-statistic):             0.0183
Time:                        03:10:20   Log-Likelihood:                -3542.6
No. Observations:                 417   AIC:                             7167.
Df Residuals:                     376   BIC:                             7333.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1561.4940     61.055     25.575      0.0

/home/ec2-user/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ec2-user/miniconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/ec2-user/miniconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/ec2-user/miniconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/ec2-user/miniconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: Runtim

In [253]:
model2 = LinearRegression()
model2.fit(X_train_sc2, y_train2)

X_test_sc2 = scaler.transform(X_test2)

preds = model.predict(X_test_sc2)

rmse = np.sqrt(mean_squared_error(y_test2, preds))
print("RMSE: %f" % (rmse))

RMSE: 1973.638555


In [61]:
params = [{'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [25, 30, 35], 'max_depth': [5, 10, 15], 
           'colsample_bytree': [0.3, 0.6, 0.9], 'objective': ['reg:linear'], 'alpha': [10]}]

clf = GridSearchCV(xgb.XGBRegressor(), params, cv=10)

clf.fit(X_train_sc, np.log(y_train))
print("Random forest classifier - Grid Search - Best params: ", clf.best_params_)
print("Random forest classifier - Grid Search - Best Score: ", clf.best_score_)

Random forest classifier - Grid Search - Best params:  {'alpha': 10, 'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 35, 'objective': 'reg:linear'}
Random forest classifier - Grid Search - Best Score:  0.15692389498974674


In [62]:
# The following set of parameters give the best performance
# {'alpha': 10, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 
#  'max_depth': 5, 'n_estimators': 25, 'objective': 'reg:linear'}
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.9, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 35)

xg_reg.fit(X_train_sc, np.log(y_train))

preds = xg_reg.predict(X_test_sc)

rmse = np.sqrt(mean_squared_error(np.log(y_test), preds))
print("RMSE: %f" % (rmse))

RMSE: 1.483349


In [260]:
raw_data

result_data = raw_data.sort_values(['ffall'], ascending=True)

In [269]:
result_data['ffall'].head(560)

2303     1
2724     1
1084     1
1562     1
643      1
1758     1
839      1
1549     1
2215     1
499      1
756      1
976      1
824      1
761      1
210      1
1534     1
817      1
2007     1
192      1
481      1
807      1
187      1
2693     1
420      1
2489     1
2626     1
180      1
2053     1
790      1
609      1
        ..
2782    16
319     16
2147    16
2208    16
2592    16
2753    16
2640    16
1037    16
874     16
2584    16
2682    16
585     16
2568    16
2571    16
2081    16
104     16
2278    16
637     16
287     16
2779    16
1640    16
657     16
248     16
465     16
814     16
1780    16
961     16
2649    16
2660    17
2582    17
Name: ffall, Length: 560, dtype: int64